# rapyuta.instr
- [photometry_profile](#photometry_profile)

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys
from pathlib import Path
import numpy as np

## Local
try:
    root = Path(__file__).parent.absolute() / '..'
except NameError:
    root = Path().absolute() / '..'
sys.path.insert(0, root)

import rapyuta.utbox as UT
from rapyuta.inout import read_hdf5, fitsext, h5ext
from rapyuta.plots import plotool, pplot
from rapyuta.instr import (
    intercalib, photometry_profile,
)

%matplotlib inline

## fold inputs
# UT.codefold(True, 'onclick')

                                                                                
                                   \  \|/  /                                    
                                \ _\_ ||| _/_ /                                 
                                      |||                                       
                                  /\/-/|\-\/\                                   
                              _ | _M_ _|_ _M_ | _                               
                            / .~.    _ ^ _    .~. \                             
                           |_|_0_|  |_&_&_|  |_0_|_|                            
                          //    .~.    o    .~.    \\                           
                         |_M_ _|_?_|__[_]__|_?_|_ _M_|                          
                                |             |                                 
                                 \  rapyuta  /                                  
                            

ImportError: cannot import name '_utils' from partially initialized module 'rapyuta.plots' (most likely due to a circular import) (/Users/dhu/Github/RAPYUTA/rapyuta/plots/__init__.py)

## photometry_profile